In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

import seaborn as sns; sns.set()

from tqdm import tqdm

chkpt_dir='../tf_ckpts'
task_name='mnist'
task = TASKS[task_name](get_task_params(), data_dir='../data')

# A Capsule Network on MNIST.
#     :param input_shape: data shape, 4d, [None, width, height, channels]
#     :param n_class: number of classes
#     :param num_routing: number of routing iterations
#     :return: A Keras Model with 2 inputs and 2 outputs
        
capsnet = MODELS['capsnet'](input_shape=[28, 28, 1],
                n_class=10,
                num_routing=3)
model.summary()

(None, 1152, 8)
Tensor("digitcaps/mul:0", shape=(None, 10, 10, 1152, 16), dtype=float32)
Tensor("digitcaps/zeros:0", shape=(None, 10, 1152), dtype=float32)


ValueError: in converted code:
    relative to /home/samira:

    Codes/InDist/tf2_models/capnet.py:155 call  *
        b += K.batch_dot(outputs, inputs_hat, [2, 3])
    anaconda3/envs/indist/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py:1824 batch_dot
        'y.shape[%d] (%d != %d).' % (axes[0], axes[1], d1, d2))

    ValueError: Cannot do batch_dot on inputs with shapes (None, 10, 10, 1152, 16) and (None, 10, 1152, 1152, 16) with axes=[2, 3]. x.shape[2] != y.shape[3] (10 != 1152).


In [ ]:
for inputs, labels in task.train_dataset:
    print(inputs.shape, labels.shape)
    plt.figure(figsize=(8,8))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(inputs[i][:,:,0], cmap=plt.cm.binary)
        plt.xlabel(labels[i].numpy())
    plt.show()
    break

In [ ]:
# Create an instance of the model
# model_params = get_model_params(task, config_name='cl_vff',model_config='ff_mnist_config')
# model = VanillaFF(model_params)

model_name = 'cl_vcnn'
model_params = get_model_params(task, config_name=model_name,model_config='vcnn_mnist')
model = MODELS[model_name](model_params)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
    
    
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  train_bz = 0
  for images, labels in task.train_dataset:
    train_step(tf.cast(images, dtype=tf.float32), tf.cast(labels, dtype=tf.int32))
    if train_bz >= task.n_train_batches:
        break
    train_bz += 1

  test_bz = 0
  for test_images, test_labels in task.test_dataset:
     test_step(tf.cast(test_images, dtype=tf.float32), tf.cast(test_labels, dtype=tf.int32))
  if test_bz >= task.n_test_batches:
        break
    test_bz += 1

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

In [ ]:
model.losses